# Проект

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
сalls — количество звонков,
minutes — суммарная длительность звонков в минутах,
messages — количество sms-сообщений,
mb_used — израсходованный интернет-трафик в Мб,
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [80]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

In [81]:
df = pd.read_csv('users_behavior.csv')

In [82]:
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [84]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Представленные данные корректы, предобработки не требуют. Пригодны для стоящей задачи бинарной классификации.

## Подготовка данных для обучения модели

In [85]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [86]:
features_train, features_test, target_train, target_test = train_test_split(
features, target, test_size=0.4, random_state=12345, stratify=target)

In [87]:
features_valid, features_test, target_valid, target_test = train_test_split(
features_test, target_test, test_size=0.5, random_state=12345, stratify=target_test)

In [88]:
print('обучающая выборка', features_train.shape)
print('валидационная выборка', features_valid.shape)
print('тестовая выборка', features_test.shape)

обучающая выборка (1928, 4)
валидационная выборка (643, 4)
тестовая выборка (643, 4)


Разделение датафрейма на известные признаки и целевой (применение тарифа "ультра"). Разделение на 3 части: 60% - обучающий набор данных, 20% - тестовый и 20% валидационный. 

In [89]:
target_train.mean()

0.3065352697095436

In [90]:
target_test.mean()

0.30637636080870917

In [91]:
target_valid.mean()

0.30637636080870917

Разделение сделано с применением параметра stratify для формирования выборок с равными долями целевого признака.

## Тест моделей классификации

In [92]:
best_model = None
best_result = 0
for depth in range(1, 6):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth = depth) 
    model_1.fit(features_train, target_train) 
    predictions_valid_1 = model_1.predict(features_valid) 
    result_1 = accuracy_score(target_valid, predictions_valid_1) 
    print(result_1, depth)
    if result_1 > best_result:
        best_model = model_1
        best_result = result_1
        best_depth = depth
print("Accuracy лучшей модели на валидационной выборке:", best_result, depth)

0.7402799377916018 1
0.7729393468118196 2
0.7776049766718507 3
0.7542768273716952 4
0.7853810264385692 5
Accuracy лучшей модели на валидационной выборке: 0.7853810264385692 5


Подбор оптимальных параметров "обучающего дерева", эмпирический подбор параметров: max_leaf_nodes, min_impurity_decrease, min_samples_leaf, min_samples_split, отличающихся от дефолтных, ухудшил показатель accuracy.

In [93]:
model_1.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 12345,
 'splitter': 'best'}

In [94]:
best_model = None
best_result = 0
for est in range(1, 100):
    model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, min_samples_split=2, min_samples_leaf=1)
    model_2.fit(features_train, target_train)
    predictions_valid_2 = model_2.predict(features_valid)
    result_2 = accuracy_score(target_valid, predictions_valid_2)
    if result_2 > best_result:
        best_model = model_2 
        best_result = result_2 

print("Accuracy наилучшей модели на валидационной выборке:", best_result, est)

Accuracy наилучшей модели на валидационной выборке: 0.8118195956454122 99


Подбор оптимальных параметров "случайного леса", так же как и с предыдущей моделью, изменение дефолтных параметров не привело к улучшению результата.

In [95]:
model_2.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 99,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 12345,
 'verbose': 0,
 'warm_start': False}

In [96]:
model_3 = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_3.fit(features_train, target_train)
result_3 = model_3.score(features_valid, target_valid)
print("Accuracy модели на валидационной выборке:", result_3)


Accuracy модели на валидационной выборке: 0.7387247278382582


In [97]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_valid, target_valid)

0.6936236391912908

## Выводы по результам исследования моделей

Лучший результат из рассмотренных моделей показала модель RandomForestClassifier на 99 деревьях. На валидационной выборке она показала точность валидации 0.8118195956454122.

## Оценка оптимальной модели на тестовой выборке

In [98]:
predictions_test_1 = model_2.predict(features_test)
result_test = accuracy_score(target_test, predictions_test_1)
print('Accuracy оптимальной модели на тестовой выборке:', result_test)

Accuracy оптимальной модели на тестовой выборке: 0.8118195956454122


Проверка оптимальной модели на тестовой выборке.

## Вывод

Для решения поставленной проблемы была сформулирована задача бинарной классификации. Были обучены, протестированы и проверены на валидационной выборке 3 модели классификации на основе "решающего дерева", "случайного леса" и логистической регрессии. Эффективность моделей оценивалась по показателю "Accuracy". Выполнена проверка на адекватность моделей. Наиболее эффективной оказалась модель RandomForestClassifier на 99 деревьях с показателем "Accuracy" = 0.812; на тестовой выборке показатель "Accuracy" не изменился. Применение этой модели поможет предложить каждому абоненту оптимальный для него тариф.